In [ ]:
from testsavant.guard import InputGuard, OutputGuard
from testsavant.guard.output_scanners import BanCode
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
def dummy_llm(input):
    return input
long_article = open("../docs/article.txt", "r").read()
print(len(long_article))

ts_api = InputGuard(
    API_KEY=os.environ.get("TEST_SAVANT_API_KEY"),
    PROJECT_ID=os.environ.get("TEST_SAVANT_PROJECT_ID"),
    remote_addr=os.environ.get("TEST_SAVANT_REMOTE_ADDR")
)
scanner = BanCode(tag="base", threshold=0.5)
ts_api.add_scanner(scanner)

In [ ]:
prompt = """
import time, os.path
photofiles = ['img_1074.jpg', 'img_1076.jpg', 'img_1077.jpg']
class BatchRename(Template):
    delimiter = '%'

fmt = input('Enter rename style (%d-date %n-seqnum %f-format):  ')


t = BatchRename(fmt)
date = time.strftime('%d%b%y')
for i, filename in enumerate(photofiles):
    base, ext = os.path.splitext(filename)
    newname = t.substitute(d=date, n=i, f=ext)
    print('{0} --> {1}'.format(filename, newname))
"""
prompts = [prompt, prompt + " " + long_article, long_article + " " + prompt]

for prompt in prompts:
    result = ts_api.scan(prompt)
    if result.is_valid:
        print("Prompt is valid")
    else:
        print("Prompt is invalid")